In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Khởi tạo trình duyệt Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
driver.get("https://www.myperfectresume.com/cv/examples")
time.sleep(5)  # Wait for initial page load
print(driver.page_source)

<html lang="en-US" prefix="og: https://ogp.me/ns#"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta http-equiv="X-UA-Compatible" content="ie=edge"><link rel="icon" type="image/x-icon" href="/favicon.ico"><link rel="preconnect" href="https://fonts.gstatic.com" crossorigin="anonymous"><link rel="preload" href="https://fonts.gstatic.com/s/nunito/v25/XRXI3I6Li01BKofiOc5wtlZ2di8HDGUmdTQ3jw.woff2" as="font" type="font/woff" crossorigin="anonymous"><link rel="preload" href="https://fonts.gstatic.com/s/nunito/v25/XRXI3I6Li01BKofiOc5wtlZ2di8HDDsmdTQ3jw.woff2" as="font" type="font/woff" crossorigin="anonymous"><link rel="preload" href="https://fonts.gstatic.com/s/domine/v19/L0xhDFMnlVwD4h3Lt9JWnbX3jG-2X5XHE1ofEw.woff2" as="font" type="font/woff" crossorigin="anonymous"><link rel="preload" href="/wp-content/themes/myperfectresume/img/myperfectresume.svg" as="image"><style>@font-face { font-family: 'Domine'; font-style: normal; font-weight: 700;

In [4]:
# Hàm cuộn trang đến cuối để tải hết nội dung
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Chờ 2 giây để tải nội dung mới
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Nếu không còn nội dung mới, thoát vòng lặp
            break
        last_height = new_height

# Cuộn trang đến cuối
print("Đang cuộn trang để tải hết nội dung...")
scroll_to_bottom()
print("Đã tải hết nội dung trang.")

Đang cuộn trang để tải hết nội dung...
Đã tải hết nội dung trang.


In [5]:
# Danh sách để lưu dữ liệu
data = []

In [9]:
try:
    # Tìm section với class "all-industry"
    all_industry_section = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "all-industry"))
    )

    # Tìm div class "container"
    container = all_industry_section.find_element(By.CLASS_NAME, "container")

    # Tìm div class "blade-content"
    blade_content = container.find_element(By.CLASS_NAME, "blade-content")

    # Tìm div class "list-items" trực tiếp trong blade-content
    list_items = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "list-items"))
    )

    # Tìm tất cả các div class "item cont-visible"
    items = list_items.find_elements(By.CLASS_NAME, "item.cont-visible")

    print(f"Tìm thấy {len(items)} phần tử 'item cont-visible'.")

    # Duyệt qua từng item
    for item in items:
        try:
            # Tìm div class "row"
            row = item.find_element(By.CLASS_NAME, "row")
            
            # Tìm tất cả các thẻ div class "eq-col" trong row
            eq_cols = row.find_elements(By.CLASS_NAME, "eq-col")
            
            # Kiểm tra xem có ít nhất 2 thẻ eq-col không
            if len(eq_cols) < 2:
                print(f"Không đủ thẻ eq-col trong item: {item.get_attribute('id')}")
                continue
            
            # Lấy thẻ eq-col thứ hai (index 1)
            eq_col = eq_cols[1]
            
            # Tìm tất cả các thẻ ul trong eq-col
            ul_tags = eq_col.find_elements(By.TAG_NAME, "ul")
            
            # Duyệt qua từng ul
            for ul in ul_tags:
                # Tìm tất cả các thẻ li trong ul
                li_tags = ul.find_elements(By.TAG_NAME, "li")
                
                # Duyệt qua từng li
                for li in li_tags:
                    try:
                        # Tìm thẻ a trong li
                        a_tag = li.find_element(By.TAG_NAME, "a")
                        
                        # Lấy link từ href
                        resume_link = a_tag.get_attribute("href")
                        
                        # Lấy nội dung của thẻ a
                        category = a_tag.text.strip()
                        
                        # Lọc bỏ các thẻ a không chứa thông tin ngành nghề (như "See all ...")
                        if "See all" in category:
                            continue
                        
                        # Thêm vào danh sách dữ liệu
                        data.append({
                            "resume_link": resume_link,
                            "Category": category
                        })
                    except Exception as e:
                        print(f"Lỗi khi xử lý thẻ li: {e}")
                        continue
        except Exception as e:
            print(f"Lỗi khi xử lý item: {e}")
            continue

except Exception as e:
    print(f"Lỗi khi truy cập cấu trúc HTML: {e}")

Tìm thấy 60 phần tử 'item cont-visible'.


In [10]:
data

[{'resume_link': 'https://www.myperfectresume.com/cv/examples/accounting/trainee',
  'Category': 'Accountant Trainee'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/accounting/officer',
  'Category': 'Accounts Officer'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/accounting/audit-assistant',
  'Category': 'Audit Assistant'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/accounting/certified-public-accountant',
  'Category': 'Certified Public Accountant'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/administrative/admin-executive',
  'Category': 'Admin Executive'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/administrative/chief-executive-officer',
  'Category': 'Administrative CEO'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/administrative/bank-clerk',
  'Category': 'Bank Clerk'},
 {'resume_link': 'https://www.myperfectresume.com/cv/examples/administrative/contract-administrator'

In [11]:
# Đóng trình duyệt
driver.quit()

In [12]:
# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(data)

In [13]:
df.head(5)

,resume_link,Category
0,https://www.myperfectresume.com/cv/examples/ac...,Accountant Trainee
1,https://www.myperfectresume.com/cv/examples/ac...,Accounts Officer
2,https://www.myperfectresume.com/cv/examples/ac...,Audit Assistant
3,https://www.myperfectresume.com/cv/examples/ac...,Certified Public Accountant
4,https://www.myperfectresume.com/cv/examples/ad...,Admin Executive


In [14]:
# Lưu vào file CSV
output_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/crawldata3links.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Dữ liệu đã được lưu vào file: {output_file}")
print(f"Tổng số link crawl được: {len(df)}")

Dữ liệu đã được lưu vào file: D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/crawldata3links.csv
Tổng số link crawl được: 245
